In [1]:
import pandas as pd
import pathlib
import numpy as np
from collections import Counter
import json
import random
import heapq

In [2]:
home_dir = pathlib.Path.home().joinpath('Documents', 'ms_thesis_ppi')
in_file = pathlib.Path.joinpath(home_dir, 'dataset','c3_data_positive.csv')
df = pd.read_csv(in_file, sep=",", dtype='unicode')
df

,int_A,int_B
0,A0A024R0Y4,Q8WWY3
1,A0A024R5S0,Q9NVV9
2,A0A024R6G0,P50222
3,A0A0C3SFZ9,A0A0S2Z5X4
4,A0A1B0GVM0,Q8N5M1
...,...,...
2545,Q81VG2,Q9Y3T9
2546,P19508,Q9Y4B6
2547,Q5NHB5,Q9Y4I1
2548,P00974,Q9Y5Y6


In [3]:
list_a = df['int_A'].tolist()
list_b = df['int_B'].tolist()
list_f = list_a + list_b
print(len(list_f))
myset = set(list_f)
print(len(myset))
seen = {}
dupes = []

for x in list_f:
    if x not in seen:
        seen[x] = 1
    else:
        if seen[x] == 1:
            dupes.append(x)
        seen[x] += 1
        
print(len(dupes))
counter = Counter(list_f)
print(counter.most_common(30))

5100
4763
337
[('A0MZ66', 2), ('A1L4F5', 2), ('A1L4G7', 2), ('A8K287', 2), ('B2RDU6', 2), ('I4AY87', 2), ('O00180', 2), ('O00204', 2), ('O00206', 2), ('O00254', 2), ('O00635', 2), ('O14618', 2), ('O14684', 2), ('O14732', 2), ('O14813', 2), ('O14829', 2), ('O14862', 2), ('O14863', 2), ('O14931', 2), ('O15217', 2), ('O15392', 2), ('O15409', 2), ('O15431', 2), ('O15455', 2), ('O43155', 2), ('O43298', 2), ('O43309', 2), ('O43312', 2), ('O43353', 2), ('O43488', 2)]


In [4]:
dict_new = dict.fromkeys(myset, 0)
df_refined = df.copy()
i = 0
indexes_to_drop = []

for index, row in df_refined.iterrows():
    partner_A = row['int_A']
    partner_B = row['int_B']
    #print(partner_A)
    if dict_new[partner_A]==0 and dict_new[partner_B]==0:
        dict_new[partner_A] += 1
        dict_new[partner_B] += 1
        i+=1
    else:
        dict_new[partner_A] += 1
        dict_new[partner_B] += 1
        indexes_to_drop.append(i)

max_value = max(dict_new.values())
print(max_value)
max_keys = [k for k, v in dict_new.items() if v == max_value]
# print(max_keys)
df_dup = df_refined[df_refined['int_A'].isin(max_keys)]
#df_refined.drop(df.index[indexes_to_drop], inplace=True )
print(df_dup)

2
       int_A   int_B
10    A0MZ66  A0MZ66
17    A1L4F5  A1L4F5
18    A1L4G7  A1L4G7
61    A8K287  A8K287
73    B2RDU6  B2RDU6
...      ...     ...
2281  Q9Y592  Q9Y592
2292  Q9Y620  Q9Y620
2294  Q9Y680  Q9Y680
2295  Q9Y6A5  Q9Y6A5
2299  Q9Y6M5  Q9Y6M5

[337 rows x 2 columns]


In [5]:
drop_list = df_dup.index.values.tolist()
# print(drop_list)
df_refined.drop(df.index[drop_list], inplace=True)
df_refined

,int_A,int_B
0,A0A024R0Y4,Q8WWY3
1,A0A024R5S0,Q9NVV9
2,A0A024R6G0,P50222
3,A0A0C3SFZ9,A0A0S2Z5X4
4,A0A1B0GVM0,Q8N5M1
...,...,...
2545,Q81VG2,Q9Y3T9
2546,P19508,Q9Y4B6
2547,Q5NHB5,Q9Y4I1
2548,P00974,Q9Y5Y6


In [6]:
# split the file into train, validation, test according to specified ratio

train_ratio, test_ratio = 0.9, 0.05
train, validate, test = np.split(df_refined.sample(frac=1), [int(train_ratio*len(df_refined)), int((train_ratio + test_ratio)*len(df_refined))])

# print("Train size_1: ", len(train_1)) # dataset for negative pair generation
print("Train size: ", len(train))
print("Val size: ", len(validate))
print("Test size", len(test))

Train size:  1991
Val size:  111
Test size 111


In [7]:
# save the dataframes to csv
train.to_csv("train_positive.csv", index=False)
#train_2.to_csv("train_positive_1.csv", index=False)
validate.to_csv("c1_val_positive.csv", index=False)
test.to_csv("test_positive.csv", index=False)
print("done")

done


In [36]:
train_file = 'train_positive.csv'
df_train = pd.read_csv(train_file, sep=",", dtype='unicode')
df_train

,int_A,int_B
0,P00451,O14832
1,O68779,Q6IEG0
2,O14828,P46934
3,O94850,Q96QZ7
4,Q15198,Q12797
...,...,...
1986,O15144,P59998
1987,O76013,Q69022
1988,A7E2Y1,Q63HR2
1989,O43752,Q12846


In [37]:
neg_left = df_train["int_A"].to_list()
neg_right = df_train["int_B"].to_list()
# neg_candidates = neg_left + neg_right
# length = len(neg_candidates)
#print(length)
rewired_left = []
rewired_right = []
i = 0

# note: we will use len(neg_candidates) as the length changes with every pop operation
while i < 20:
    neg_candidates = neg_left + neg_right
    for p in range(len(neg_candidates) // 2):
        rewired_left.append(neg_candidates.pop(random.randrange(len(neg_candidates))))
        rewired_right.append(neg_candidates.pop(random.randrange(len(neg_candidates))))
    i+=1

print(len(rewired_left))
print(len(rewired_right))

39820
39820


In [38]:
df_neg = pd.DataFrame()
df_neg['int_A'] = rewired_left
df_neg['int_B'] = rewired_right
df_neg.to_csv("negative_pairs.csv", index=False)

In [39]:
df_neg.drop_duplicates(subset=['int_A', 'int_B'])
df_neg

,int_A,int_B
0,P52747,Q9HB58
1,Q96DI7,Q99519
2,Q8CK76,Q12866
3,Q8D0V9,Q15637
4,Q08378,Q8D0H2
...,...,...
39815,P35790,P26006
39816,P14317,Q15326
39817,Q8NA59,P02671
39818,O96007,Q8ZIY9


In [11]:
in_file_2 = pathlib.Path.joinpath(home_dir, 'dataset','9606_Q1.txt')
df_whole = pd.read_csv(in_file_2, sep="\t", header=0, dtype='unicode')
df_whole = df_whole[['UniprotID_A', 'UniprotID_B']] # only keep specific columns
df_whole.columns = ["int_A", "int_B"]
#df_whole

In [43]:
comparison_df = df_neg.merge(df_whole, indicator=True, how='outer')
neg_train_final = comparison_df.loc[comparison_df['_merge'] == 'left_only']
neg_train_final.to_csv("neg_train_final.csv", index=False)
neg_train_final

,int_A,int_B,_merge
0,P52747,Q9HB58,left_only
1,Q96DI7,Q99519,left_only
2,Q8CK76,Q12866,left_only
3,Q8D0V9,Q15637,left_only
4,Q08378,Q8D0H2,left_only
...,...,...,...
39815,P35790,P26006,left_only
39816,P14317,Q15326,left_only
39817,Q8NA59,P02671,left_only
39818,O96007,Q8ZIY9,left_only


In [8]:
test_file = 'c1_val_positive.csv'
df_test = pd.read_csv(test_file, sep=",", dtype='unicode')
df_test

,int_A,int_B
0,Q8WZ59,Q07108
1,Q8N720,Q8D0K7
2,O54946,Q13686
3,P58497,Q99460
4,Q9BUB5,Q99615
...,...,...
106,O15304,Q8ZJP7
107,O15354,Q9Y3R0
108,P68363,P0C7T8
109,Q5NI89,Q5U651


In [9]:
# neg test set generation

neg_left = df_test["int_A"].to_list()
neg_right = df_test["int_B"].to_list()
# neg_candidates = neg_left + neg_right
# length = len(neg_candidates)
#print(length)
rewired_left = []
rewired_right = []
i = 0

# note: we will use len(neg_candidates) as the length changes with every pop operation
while i < 20:
    neg_candidates = neg_left + neg_right
    for p in range(len(neg_candidates) // 2):
        rewired_left.append(neg_candidates.pop(random.randrange(len(neg_candidates))))
        rewired_right.append(neg_candidates.pop(random.randrange(len(neg_candidates))))
    i+=1

print(len(rewired_left))
print(len(rewired_right))
df_neg_t = pd.DataFrame()
df_neg_t['int_A'] = rewired_left
df_neg_t['int_B'] = rewired_right
df_neg_t.to_csv("neg_test_final.csv", index=False)
df_neg_t.drop_duplicates(subset=['int_A', 'int_B'])
df_neg_t

2220
2220


,int_A,int_B
0,Q81MB3,Q6EMK4
1,P28062,Q96RT1
2,A6NC98,Q5W5X9
3,Q8N5C8,Q8CZW4
4,P49959,Q93052
...,...,...
2215,P46736,A3KN83
2216,P62318,O95273
2217,Q99615,Q81WR4
2218,Q9UKY1,O75436


In [ ]:
# save negative validation set
comparison_df_v = df_neg_t.merge(df_whole, indicator=True, how='outer')
neg_test_final = comparison_df_v.loc[comparison_df_v['_merge'] == 'left_only']
neg_test_final.to_csv("c1_neg_val_final.csv", index=False)

In [16]:
neg_c1 = neg_test_final[['int_A', 'int_B']]
c1_test_final = df_test.append(neg_c1, ignore_index=True)
c1_test_final.to_csv("c1_test.csv", index=False)

In [20]:
ID = {}
label_pos = {}
labels = c1_test_final.index.values
max_label = 0
for i in range(len(labels)):
    temp_list = [df.at[i, 'int_A'], df.at[i, 'int_B']]
    ID[str(labels[i])] = temp_list
    if i < 111:
        label_pos[str(labels[i])] = 1
    else:
        label_pos[str(labels[i])] = 0
    max_label = i+1

In [24]:
import json

def save_json(filename, data):
    with open(filename, 'w') as fp:
        json.dump(data, fp)

save_json('c1_test_pairs.json', ID)
save_json('c1_test_labels.json', label_pos)

In [13]:
test1 = df['int_A'].tolist()
test2 = df['int_B'].tolist()
test = test1 + test2
with open("prot_list.txt", 'w') as output:
    for row in test:
        output.write(str(row) + '\n')
print(len(test))    

5100


In [17]:
freq = []
with open('bind_frequency.json') as json_file:
    data = json.load(json_file)
    print("Type:", type(data))
    for i in test:
        freq.append(data[i])

heapq.nlargest(30, freq)
print(list(data.keys())[list(data.values()).index(630)])

Type: <class 'dict'>
P19320
